### Actions and Msgs

In [1]:
def combine_processing_actions_and_msgs(url='/Users/anna/Downloads/ngwl-predict-customer-churn/messages/'):
    import pandas as pd
    import numpy as np
    from tqdm.notebook import tqdm

    # pip install emoji
    #import emoji

    # открываем данные
    actions = pd.read_csv(url + 'actions.csv')
    messages = pd.read_csv(url + 'messages.csv')

    # переводим в общим формат с actions id
    messages = messages.astype({'action_id': 'int64'})

    # работаем со временем
    messages['sent'] = pd.to_datetime(messages['sent'], unit='s')
    ActM_combine = messages.merge(
        actions, left_on=['action_id'], right_on=['id'])
    ActM_combine['date'] = ActM_combine['sent'].dt.date
    date = pd.to_datetime(ActM_combine['date'])
    ActM_combine['date'] = date
    year = ActM_combine.date.dt.year
    month = ActM_combine.date.dt.month
    day = ActM_combine.date.dt.day
    wday = ActM_combine.date.dt.dayofweek

    # вытаскиваем дни недели и тд
    ActM_combine['date'] = date
    ActM_combine['year'] = year
    ActM_combine['month'] = month
    ActM_combine['day'] = day
    ActM_combine['wday'] = wday

    hours = ActM_combine['sent'].dt.hour
    minutes = ActM_combine['sent'].dt.minute
    ActM_combine['hours'] = hours
    ActM_combine['minutes'] = minutes

    # дроп ненужное
    ActM_combine = ActM_combine.drop(['date'], axis=1)
    ActM_combine = ActM_combine.drop(['sent'], axis=1)
    ActM_combine = ActM_combine.drop(['id'], axis=1)

    # ЛЭ для эмеил/сообщение/пуш-уведомление
#     from sklearn import preprocessing

#     type_msg = ActM_combine['type'].to_frame()
#     type_msg_le = type_msg.apply(preprocessing.LabelEncoder().fit_transform)
#     ActM_combine['type'] = type_msg_le
#     dummy = pd.get_dummies(ActM_combine['type'])
#     ActM_combine = pd.concat([ActM_combine, dummy], axis = 0)

    ActM_combine.rename(columns={"year": "year_act",
                                 "month": "month_act",
                                 "day": "day_act",
                                 "wday": "wday_act",
                                 "hours": "hours_act",
                                 "minutes": "minutes_act", })

    # переводим все в нумпай, чтобы быстрее обрабатывать данные
    array = ActM_combine.values

    # заменим все пропущенные значения в пушах на 0
    ActM_combine.subject.fillna(0, inplace=True)
    

    # поиск "промокод" в списке + дата до какого действует промокод
    array_promocode = []
    array_promocode_data = []

    for i in tqdm(range(len(array))):
        each_str = array[i][3]

        try:
            each_str = each_str.replace('#', '').replace('@', '').replace(
                '!', '').replace(',', '').replace('.', '')
            each_str = each_str.lower().split(' ')

            if 'промокод' in each_str:
                array_promocode.append(1)
                if 'до' in each_str:
                    for j in range(len(each_str)):
                        if each_str[j] == 'до':
                            array_promocode_data.append(each_str[j+1])
                else:
                    array_promocode_data.append(0)
            else:
                array_promocode.append(0)
                array_promocode_data.append(0)

        except:
            array_promocode.append(0)
            array_promocode_data.append(0)

    subject_promotion = []  # акции
    subject_other = []  # другое
    subject_shopping_cart = []  # корзина
    subject_discount = []  # скидки
    subject_bonus = []  # бонусы

    for i in tqdm(range(len(array))):
        each_str = array[i][2]

        try:
            each_str = each_str.replace('#', '').replace('@', '').replace(
                '!', '').replace(',', '').replace('.', '').replace('?', '')
            each_str = each_str.lower().split(' ')

            cart = 0
            discount = 0
            bonus = 0
            promotion = 0
            other = 0

            if 'корзине' in each_str:
                cart = 1
            elif 'скидки' in each_str or 'скидка' in each_str or 'скидку' in each_str:
                discount = 1
            elif 'бонусов' in each_str or 'бонусы' in each_str:
                bonus = 1
            elif 'акция' in each_str:
                promotion = 1
            else:
                other = 1

            subject_shopping_cart.append(cart)
            subject_discount.append(discount)
            subject_bonus.append(bonus)
            subject_promotion.append(promotion)
            subject_other.append(other)

        except:
            subject_promotion.append(0)
            subject_other.append(0)
            subject_shopping_cart.append(0)
            subject_discount.append(0)
            subject_bonus.append(0)

    # записываем новые колонки в датасет
    ActM_combine['promocode'] = array_promocode
    ActM_combine['promocode_data'] = array_promocode_data
    ActM_combine['shopping_cart'] = subject_shopping_cart
    ActM_combine['discount'] = subject_discount
    ActM_combine['bonus'] = subject_bonus
    ActM_combine['promotion'] = subject_promotion
    ActM_combine['other'] = subject_other

    ActM_combine['promo_data_day'] = 0
    ActM_combine['promo_data_month'] = 0
    for i in tqdm(range(len(ActM_combine))):
        if ActM_combine['promocode_data'][i] != 0:
            ActM_combine['promo_data_day'][i] = int(ActM_combine['promocode_data'][i][0] +
                                                    ActM_combine['promocode_data'][i][1])
            ActM_combine['promo_data_month'][i] = int(ActM_combine['promocode_data'][i][2] +
                                                      ActM_combine['promocode_data'][i][3])

    ActM_combine = ActM_combine.drop(['promocode_data'], axis=1)

    # сколько push уведомлений приходит человеку

    a = ActM_combine.user_id.value_counts()
    a = dict(a)
    a = pd.DataFrame({'count': a}, columns=['count']).reset_index()
    ActM_combine = ActM_combine.merge(a, left_on='user_id', right_on='index')
    ActM_combine = ActM_combine.drop(['index'], axis=1)

    return ActM_combine

In [2]:
ActM_combine = combine_processing_actions_and_msgs()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
#ActM_combine.to_csv('ActM_combine.csv')

________

________

______

In [25]:
def months_processing(df=ActM_combine):
    import pandas as pd
    from tqdm.notebook import tqdm

    df_copy = ActM_combine.copy()
    
    
    df_copy.body.fillna('empty', inplace=True)
    
    
    df_copy_array = df_copy.values
    
    find_empty_mess = []
    for i in tqdm(range(len(df_copy_array))):
        subject_str = df_copy_array[i][2]
        body_str = df_copy_array[i][3]

        if subject_str == '0' and body_str == 'empty':
            find_empty_mess.append(1)
        else:
            find_empty_mess.append(0)

    df_copy['empty_msgs'] = find_empty_mess

    dummy = pd.get_dummies(df_copy['type'])
    df_copy['email'] = dummy['email']
    df_copy['push'] = dummy['push']
    df_copy['sms'] = dummy['sms']
    df_copy = df_copy.drop(['subject', 'body', 'type', 'day',
                            'wday', 'hours', 'minutes', 'count'], axis=1)

    main_dataframe = pd.DataFrame()

    print(df_copy['year'].unique())

    for time_season in tqdm(range(1, 12)):
        df_season = df_copy[df_copy['month'] == time_season]
        df_season = df_season.drop(['action_id', 'year', 'month'], axis=1)
        df_season = df_season.groupby(['user_id'], as_index=False).sum()
        df_season['month'] = time_season
        main_dataframe = main_dataframe.append(df_season)

    return main_dataframe

In [26]:
main_dataframe = months_processing(ActM_combine)


[2020]


In [27]:
main_dataframe

,user_id,promocode,shopping_cart,discount,bonus,promotion,other,promo_data_day,promo_data_month,empty_msgs,email,push,sms,month
0,1400,0,0,2,0,0,4,0,0,1,0,7,0,1
1,1670,0,0,1,0,0,3,0,0,0,1,3,0,1
2,1833,0,0,0,0,0,1,0,0,0,1,0,0,1
3,1940,0,0,0,0,0,1,0,0,0,0,1,0,1
4,2151,0,0,4,0,0,3,0,0,1,0,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450545,2925484,0,0,0,0,0,1,0,0,0,1,0,0,8
450546,2925486,0,0,0,0,0,1,0,0,0,1,0,0,8
450547,2925490,0,0,0,0,0,2,0,0,0,0,2,0,8
450548,2925494,0,0,0,0,0,3,0,0,5,1,7,0,8


In [28]:
main_dataframe.to_csv('main_dataframe.csv')